In [ ]:
!pip install --no-deps bitsandbytes accelerate  peft trl transformers
!pip install sentencepiece  datasets huggingface_hub 
!pip install flash-attn --no-build-isolation

In [ ]:
import os
import torch
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict
from peft import AdaLoraConfig, get_peft_model
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
import logging
import gc

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Set seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

In [ ]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.
model_name = "meta-llama/Llama-3.1-8B-Instruct"

In [ ]:

df = pd.read_csv("/content/Data.csv")  


dataset = Dataset.from_pandas(df)
train_test_dataset = dataset.train_test_split(test_size=0.1, seed=42)
train_val_dataset = DatasetDict({
    'train': train_test_dataset['train'],
    'validation': train_test_dataset['test']
})


quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,  
    bnb_4bit_quant_type="nf4",      
)